In [ ]:
using Random
using Plots
using ProgressMeter
using Colors

f(x, y) = 20 + (x^2 - 10*cos(2π*x)) + (y^2 - 10*cos(2π*y))
# f(x, y) = (x-3)^2 + (y-3)^2

function decimal_to_binary(x::Int, length::Int)
    binary_str = bitstring(x)[end-length+1:end]
    return binary_str
end

function binary_to_decimal(binary_str::String)
    return parse(Int, binary_str, base=2)
end

function calculate_fitness(population::Vector{Tuple{String, String}}, fitness_func)
    fitness_values = Float64[]
    decimal_values_x = Int[]
    decimal_values_y = Int[]

    for (chrom_x, chrom_y) in population
        x = binary_to_decimal(chrom_x)
        y = binary_to_decimal(chrom_y)
        push!(decimal_values_x, x)
        push!(decimal_values_y, y)
        push!(fitness_values, fitness_func(x, y))
    end
    return fitness_values, decimal_values_x, decimal_values_y
end

function select_parents(population::Vector{Tuple{String, String}}, fitness_values::Vector{Float64})
    inverted_fitness = maximum(fitness_values) .- fitness_values .+ 1e-10
    total_fitness = sum(inverted_fitness)
    probabilities = inverted_fitness ./ total_fitness

    parent1_idx = findfirst(cumsum(probabilities) .≥ rand())[1]
    parent2_idx = findfirst(cumsum(probabilities) .≥ rand())[1]

    return population[parent1_idx], population[parent2_idx]
end

function crossover(parent1::Tuple{String, String}, parent2::Tuple{String, String}, crossover_rate::Float64)
    if rand() > crossover_rate
        return parent1, parent2
    end

    crossover_point_x = rand(1:length(parent1[1])-1)
    child1_x = parent1[1][1:crossover_point_x] * parent2[1][crossover_point_x+1:end]
    child2_x = parent2[1][1:crossover_point_x] * parent1[1][crossover_point_x+1:end]

    crossover_point_y = rand(1:length(parent1[2])-1)
    child1_y = parent1[2][1:crossover_point_y] * parent2[2][crossover_point_y+1:end]
    child2_y = parent2[2][1:crossover_point_y] * parent1[2][crossover_point_y+1:end]

    return (child1_x, child1_y), (child2_x, child2_y)
end

function mutate(child::Tuple{String, String}, mutation_rate::Float64)
    if rand() <= mutation_rate
        mutation_point_x = rand(1:length(child[1]))
        mutated_child_x = collect(child[1])
        mutated_child_x[mutation_point_x] = mutated_child_x[mutation_point_x] == '0' ? '1' : '0'
        child_x = join(mutated_child_x)
    else
        child_x = child[1]
    end

    if rand() <= mutation_rate
        mutation_point_y = rand(1:length(child[2]))
        mutated_child_y = collect(child[2])
        mutated_child_y[mutation_point_y] = mutated_child_y[mutation_point_y] == '0' ? '1' : '0'
        child_y = join(mutated_child_y)
    else
        child_y = child[2]
    end

    return (child_x, child_y)
end

function plot_generation_3d(x_values::Vector{Int}, y_values::Vector{Int}, fitness_values::Vector{Float64},
                           generation::Int, x_range::Tuple{Int,Int}, y_range::Tuple{Int,Int},
                           best_x::Int, best_y::Int)

    x_plot = range(x_range[1], x_range[2], length=100)
    y_plot = range(y_range[1], y_range[2], length=100)
    z = [f(x, y) for x in x_plot, y in y_plot]
    
    z_values = [f(x, y) for (x, y) in zip(x_values, y_values)]
    best_z = f(best_x, best_y)

    p = plot(x_plot, y_plot, z, st=:surface, alpha=0.5, color=:viridis,
             title="Поколение $generation", xlabel="x", ylabel="y", zlabel="f(x,y)",
             xlims=x_range, ylims=y_range)

    scatter!(x_values, y_values, z_values, color=:red, markersize=5,
             label="Популяция", marker=:circle)

    scatter!([best_x], [best_y], [best_z], color=:green, markersize=8,
             label="Лучшая точка", marker=:star5)

    return p
end

function genetic_algorithm_with_animation_3d(fitness_func, population_size::Int,
                                           chromosome_length::Int, generations::Int,
                                           crossover_rate::Float64, mutation_rate::Float64,
                                           x_range::Tuple{Int,Int}, y_range::Tuple{Int,Int},
                                           gif_filename::String)

    min_x, max_x = x_range
    min_y, max_y = y_range

    initial_population = [(decimal_to_binary(rand(min_x:max_x), chromosome_length),
                           decimal_to_binary(rand(min_y:max_y), chromosome_length))
                          for _ in 1:population_size]

    current_population = copy(initial_population)
    animation = Animation()
    best_x = 0
    best_y = 0
    best_fitness = Inf

    @showprogress for gen in 1:generations
        fitness_values, decimal_values_x, decimal_values_y = calculate_fitness(current_population, fitness_func)

        current_best_idx = argmin(fitness_values)
        if fitness_values[current_best_idx] < best_fitness
            best_x = decimal_values_x[current_best_idx]
            best_y = decimal_values_y[current_best_idx]
            best_fitness = fitness_values[current_best_idx]
        end

        p = plot_generation_3d(decimal_values_x, decimal_values_y, fitness_values, gen, x_range, y_range, best_x, best_y)
        frame(animation, p)

        println("\nПоколение $gen:")
        println("Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность")
        println("-----------|------------|------------|------------|------------------")
        for (chrom_x, chrom_y, x, y, fit) in zip(
            getindex.(current_population, 1),
            getindex.(current_population, 2),
            decimal_values_x, decimal_values_y, fitness_values
        )
            println("  $chrom_x  |   $chrom_y  |     $x     |     $y     |     $fit")
        end

        new_population = Vector{Tuple{String, String}}()

        while length(new_population) < population_size
            parent1, parent2 = select_parents(current_population, fitness_values)

            child1, child2 = crossover(parent1, parent2, crossover_rate)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            push!(new_population, child1)
            if length(new_population) < population_size
                push!(new_population, child2)
            end
        end

        current_population = new_population
    end

    gif(animation, gif_filename, fps=1)

    final_fitness, final_decimal_x, final_decimal_y = calculate_fitness(current_population, fitness_func)
    best_idx = argmin(final_fitness)
    best_chrom_x = current_population[best_idx][1]
    best_chrom_y = current_population[best_idx][2]
    best_x = final_decimal_x[best_idx]
    best_y = final_decimal_y[best_idx]
    best_fitness = final_fitness[best_idx]

    println("\nРезультат:")
    println("Лучшая хромосома X: $best_chrom_x")
    println("Лучшая хромосома Y: $best_chrom_y")
    println("Десятичное значение X: $best_x")
    println("Десятичное значение Y: $best_y")
    println("Приспособленность: $best_fitness")

    return (best_x, best_y)
end

population_size = 8
chromosome_length = 5
generations = 200
crossover_rate = 0.8
mutation_rate = 0.1
x_range = (0, 10)
y_range = (0, 10)
gif_filename = "genetic_algorithm_animation_3d.gif"

best_solution = genetic_algorithm_with_animation_3d(
    f, population_size, chromosome_length, generations,
    crossover_rate, mutation_rate, x_range, y_range, gif_filename
)

println("\nАнимация сохранена в файл: $gif_filename")
println("Лучшее решение: x = $(best_solution[1]), y = $(best_solution[2])")


Поколение 1:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00110  |   01001  |     6     |     9     |     117.0
  00111  |   00111  |     7     |     7     |     98.0
  00001  |   00101  |     1     |     5     |     26.0
  00101  |   01000  |     5     |     8     |     89.0
  01010  |   00110  |     10     |     6     |     136.0
  01000  |   00000  |     8     |     0     |     64.0
  00011  |   01010  |     3     |     10     |     109.0
  00100  |   00111  |     4     |     7     |     65.0

Поколение 2:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00001  |   01000  |     1     |     8     |     65.0
  00101  |   00101  |     5     |     5     |     50.0
  00101  |   01000  |     5     |     8     |     89.0
  00111  |   00111  |     7     |     7     |     98.0
  00001  |   01101  |  

Progress:   1%|█                                        |  ETA: 0:04:12



Поколение 3:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00111  |   00110  |     7     |     6     |     85.0
  00011  |   01011  |     3     |     11     |     130.0
  00011  |   00111  |     3     |     7     |     58.0
  00111  |   01010  |     7     |     10     |     149.0
  00111  |   00111  |     7     |     7     |     98.0
  00100  |   00111  |     4     |     7     |     65.0
  00001  |   01000  |     1     |     8     |     65.0
  00001  |   01000  |     1     |     8     |     65.0

Поколение 4:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00101  |   00000  |     5     |     0     |     25.0
  00000  |   01111  |     0     |     15     |     225.0
  00001  |   00000  |     1     |     0     |     1.0
  00011  |   01111  |     3     |     15     |     234.0
  00100  |   00111  

Progress:   5%|███                                      |  ETA: 0:01:11



Поколение 9:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00010  |   00100  |     2     |     4     |     20.0
  00000  |   00001  |     0     |     1     |     1.0
  01011  |   00000  |     11     |     0     |     121.0
  00000  |   00100  |     0     |     4     |     16.0
  00000  |   00100  |     0     |     4     |     16.0
  01000  |   00000  |     8     |     0     |     64.0
  00010  |   00001  |     2     |     1     |     5.0
  00000  |   00100  |     0     |     4     |     16.0

Поколение 10:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00000  |   00001  |     0     |     1     |     1.0
  00000  |   10100  |     0     |     20     |     400.0
  01000  |   00001  |     8     |     1     |     65.0
  00010  |   00000  |     2     |     0     |     4.0
  00000  |   00100  |     

...

Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:00


Поколение 199:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00000  |   00010  |     0     |     2     |     4.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   10000  |     0     |     16     |     256.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0


Progress: 100%|█████████████████████████████████████████| Time: 0:00:24



Поколение 200:
Хромосома X | Хромосома Y | Значение X | Значение Y | Приспособленность
-----------|------------|------------|------------|------------------
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0
  00001  |   00000  |     1     |     0     |     1.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   10000  |     0     |     16     |     256.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   00000  |     0     |     0     |     0.0
  00000  |   10000  |     0     |     16     |     256.0

Результат:
Лучшая хромосома X: 00000
Лучшая хромосома Y: 00000
Десятичное значение X: 0
Десятичное значение Y: 0
Приспособленность: 0.0

Анимация сохранена в файл: genetic_algorithm_animation_3d.gif
Лучшее решение: x = 0, y = 0


┌ Info: Saved animation to c:\Users\Golum\Desktop\iu9-education\optimization-methods\lab11\genetic_algorithm_animation_3d.gif
└ @ Plots C:\Users\Golum\.julia\packages\Plots\kLeqV\src\animation.jl:156
